# ___Feature Engineering___
-----------------

In [1]:
import numpy as np
np.random.seed(2024 - 5 - 14)
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from scipy.stats import pearsonr

import matplotlib.pyplot as plt

In [1]:
# not all features in a dataset are equally informative for the ML task we are interested in
# imagine a house sales data that has columns - land_width, land_length, sales_price
# using these two features, we could build a linear regression model like 

# ___$f_{\overrightarrow{w}, b}(\overrightarrow{x}) = w_1x_1 + w_2x_2 + b$___

In [5]:
# we could compute the land area by land_width x land_length and use it as a replacement for the two separate features length and width
# because in a sales context, land area is a more influential predictor than width and length of the land!
# so we could reengineer the model like this

# ___$f_{w, b}(x) = wx + b$___

In [6]:
# or we could introduce the area as a third predictor without removing the first two predictors, like so

# ___$f_{\overrightarrow{w}, b}(\overrightarrow{x}) = w_1x_1 + w_2x_2 + w_3x_3 + b$___

In [7]:
# FEATURE ENGINEERING OFTEN REQUIRES SOME KNOWLEDGE ABOUT THE DATA AND USE OF ONE'S EXPERIENCE AND INTUITION

# ___Ames, Iowa Housing dataset___
-----------------

In [135]:
# https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt
# http://ww2.amstat.org/publications/jse/v19n3/decock.pdf

In [6]:
ames = pd.read_csv(r"./ames_house_prices.csv", na_values = '?', index_col = 0).reset_index().drop("Id", axis = 1)
ames.dropna(axis = 1, inplace = True)

In [11]:
target = ames.SalePrice
ames.drop("SalePrice", axis = 1, inplace = True)

In [15]:
ames.columns

Index(['MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')

## ___Without feature engineering___
---------------------

In [53]:
np.unique(ames.dtypes, return_counts = True)

(array([dtype('int64'), dtype('O')], dtype=object),
 array([33, 27], dtype=int64))

In [57]:
model = LinearRegression(fit_intercept = True, n_jobs = 16)
scaler = StandardScaler(with_mean = True, with_std = True)

In [55]:
# using label encoding for categorical data in regression problems is a bad idea!

In [56]:
ames.select_dtypes(object)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,TA,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,'Wd Sdng','Wd Shng',TA,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,GasA,Ex,Y,TA,Typ,Y,WD,Normal
1456,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,TA,TA,CBlock,GasA,TA,Y,TA,Min1,Y,WD,Normal
1457,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,Ex,Gd,Stone,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1458,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,TA,TA,CBlock,GasA,Gd,Y,Gd,Typ,Y,WD,Normal


In [52]:
pd.get_dummies(ames.select_dtypes(object)).astype(np.int32)

,MSZoning_'C (all)',MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_'Wd Sdng',Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_WdShing,Exterior2nd_'Brk Cmn',Exterior2nd_'Wd Sdng',Exterior2nd_'Wd Shng',Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [65]:
int64_colnames = ames.select_dtypes(np.int64).columns

In [70]:
int64_cols = pd.DataFrame(scaler.fit_transform(ames.select_dtypes(np.int64)), columns = int64_colnames)

In [74]:
# pd.get_dummies(ames.select_dtypes(object)).astype(np.int32)

In [77]:
predictors = pd.concat([int64_cols, pd.get_dummies(ames.select_dtypes(object)).astype(np.int32)], axis = 1)

In [83]:
train_x, test_x, train_y, test_y = train_test_split(predictors, target, train_size = 0.85, random_state = 0)

In [84]:
model.fit(train_x, train_y)

LinearRegression(n_jobs=16)

In [85]:
preds = model.predict(test_x)

In [87]:
pearsonr(test_y, preds)

PearsonRResult(statistic=-0.05507735574130762, pvalue=0.41735316592774185)

In [88]:
# that's phenomenally terrible

In [89]:
# using label encoding instead of categorical feature expansion

In [107]:
ames = pd.read_csv(r"./ames_house_prices.csv", na_values = '?', index_col = 0).reset_index().drop("Id", axis = 1)
ames.dropna(axis = 1, inplace = True)

In [108]:
model = LinearRegression(fit_intercept = True, n_jobs = 16)
scaler = MinMaxScaler()
encoder = LabelEncoder()

In [109]:
target = ames.SalePrice
ames.drop("SalePrice", axis = 1, inplace = True)

In [110]:
encoded_objcolumns = ames.select_dtypes(object).apply(lambda column: encoder.fit_transform(column))

In [111]:
encoded_predictors = pd.concat([ames.select_dtypes(np.int64), encoded_objcolumns], axis = 1)

In [112]:
scaled_x = scaler.fit_transform(encoded_predictors)

In [118]:
train_x, test_x, train_y, test_y = train_test_split(scaled_x, target, train_size = 0.85, random_state = 1000)

In [119]:
model.fit(train_x, train_y)

LinearRegression(n_jobs=16)

In [120]:
preds = model.predict(test_x)

In [121]:
pearsonr(preds, test_y)

PearsonRResult(statistic=0.9291619044045185, pvalue=9.577733935645116e-96)

In [ ]:
# that's really good but how??

In [131]:
coeffs = pd.Series(model.coef_, index = encoded_predictors.columns)

In [134]:
coeffs.sort_values()

GrLivArea       -5.701741e+17
BsmtFinSF1      -2.804966e+17
BsmtUnfSF       -1.160950e+17
BsmtFinSF2      -7.325513e+16
Utilities       -5.842400e+04
Condition2      -5.078400e+04
BedroomAbvGr    -4.010111e+04
KitchenQual     -3.558400e+04
KitchenAbvGr    -3.179233e+04
ExterQual       -3.154400e+04
MSSubClass      -1.373207e+04
PoolArea        -1.234500e+04
GarageArea      -1.130221e+04
OpenPorchSF     -1.128761e+04
HouseStyle      -1.030400e+04
BldgType        -1.017600e+04
Exterior1st     -7.488000e+03
MoSold          -7.046026e+03
HalfBath        -6.381364e+03
LotShape        -5.200000e+03
Condition1      -4.944000e+03
MSZoning        -4.128000e+03
Heating         -3.952000e+03
YrSold          -3.859056e+03
YearRemodAdd    -3.693156e+03
HeatingQC       -3.264000e+03
SaleType        -1.056000e+03
CentralAir      -8.320000e+02
ExterCond       -5.600000e+02
LotConfig       -4.160000e+02
PavedDrive       6.400000e+01
Foundation       1.268000e+03
Exterior2nd      4.800000e+03
BsmtHalfBa

## ___With feature engineering___
------------------

In [33]:
# let's hand pick predictors

ames.drop(["MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition", "1stFlrSF", "2ndFlrSF", "Condition1", "Condition2", "MSZoning"], axis = 1)

,MSSubClass,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea
0,60,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0
1,20,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0
2,60,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0
3,70,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,1Fam,2Story,7,5,1915,1970,Gable,CompShg,'Wd Sdng','Wd Shng',TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0
4,60,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,0,0,953,953,GasA,Ex,Y,0,1647,0,0,2,1,3,1,TA,7,Typ,1,2,460,Y,0,40,0,0,0,0
1456,20,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,TA,TA,CBlock,790,163,589,1542,GasA,TA,Y,0,2073,1,0,2,0,3,1,TA,7,Min1,2,2,500,Y,349,0,0,0,0,0
1457,70,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,Ex,Gd,Stone,275,0,877,1152,GasA,Ex,Y,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,1,252,Y,0,60,0,0,0,0
1458,20,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,TA,TA,CBlock,49,1029,0,1078,GasA,Gd,Y,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,1,240,Y,366,0,112,0,0,0


In [32]:
ames.MSZoning.unique()

array(['RL', 'RM', "'C (all)'", 'FV', 'RH'], dtype=object)